1 - INSTALLAZIONE DELLE LIBRERIE

In [2]:
!pip install -U keras
!pip install -q tensorflow_datasets
!pip install -U tensorflow
!pip install tensorflow-examples
!pip install numpy==1.26.0
!pip install pycocotools
!pip install scikit-image
!pip install pyntcloud
!pip install torch
!pip install tensorflow-hub

2 - IMPORT 

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix
from IPython.display import clear_output
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import cv2  
import os
import ijson
import json
import urllib.request
import zipfile
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import SeparableConv2D, Conv2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Model, load_model
from skimage.filters import median
from skimage.morphology import disk
import plotly.graph_objs as go
import tarfile
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
import keras
from tkinter import Tk, filedialog
from keras.saving import register_keras_serializable
import random
import shutil


3 - DIRECTORY DEL PROGETTO DA SOSTITUIRE CON LA PROPRIA CARTELLA

In [4]:
#DIRECTORY DEL PROGETTO DA SOSTITUIRE 
project_dir = 'C:/Users/Tesisti/Desktop/CalabreseCarretta/Progetto'
#VARIABILE PER ESEGUIRE L'ADDESTRAMENTO
addestramento = False

4 - DOWNLOAD DEL DATASET DI TRAINING

In [5]:
def download_and_extract(url, download_dir, extract_to=None):
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    filename = url.split("/")[-1]
    file_path = os.path.join(download_dir, filename)

    # Scarica il file se non esiste già
    if not os.path.exists(file_path):
        print(f"Scaricando {filename}...")
        urllib.request.urlretrieve(url, file_path)
        print(f"Scaricato {filename} in {file_path}")
    else:
        print(f"{filename} esiste già, non è necessario scaricare.")

    # Estrai il file ZIP solo se necessario
    if file_path.endswith(".zip") and extract_to:
        # Verifica se la cartella è vuota
        if not os.path.exists(extract_to):
            if not os.path.exists(extract_to):
                os.makedirs(extract_to)  # Crea la directory di estrazione
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                print(f"Estrazione di {filename} in {extract_to}...")
                # Estrai ogni file nella directory di destinazione senza creare sottocartelle duplicate
                for member in zip_ref.namelist():
                    member_path = os.path.join(extract_to, os.path.basename(member))
                    if os.path.basename(member):  # Ignora eventuali directory vuote
                        with open(member_path, "wb") as output_file:
                            output_file.write(zip_ref.read(member))
                print(f"Estrazione completata.")
        else:
            print(f"I file in {extract_to} esistono già. Saltando l'estrazione.")
    return file_path


# Percorso di salvataggio
train_dataset_dir = os.path.join(project_dir, 'COCOdataset2017')
images_dir = os.path.join(train_dataset_dir, 'images')
annotations_dir = os.path.join(train_dataset_dir, 'annotations')

train_images_dir = os.path.join(images_dir, 'train2017')
val_images_dir = os.path.join(images_dir, 'val2017')
test_images_dir = os.path.join(images_dir, 'test')
# Verifica se il dataset è già presente
if not os.path.exists(train_dataset_dir):
    # URL per scaricare i file
    urls = {
        "train_images": "http://images.cocodataset.org/zips/train2017.zip",
        "val_images": "http://images.cocodataset.org/zips/val2017.zip",
        "annotations": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
    }

    # Scarica le immagini di training
    
    download_and_extract(urls['train_images'], train_dataset_dir, images_dir)

    # Scarica le immagini di validazione (che verranno usate come test)
    
    download_and_extract(urls['val_images'], train_dataset_dir, images_dir)

    # Rinominare la cartella val2017 in test
    
    if os.path.exists(val_images_dir):
        os.rename(val_images_dir, test_images_dir)
        print(f"Cartella {val_images_dir} rinominata in {test_images_dir}.")

    # Creazione del dataset di validazione (20% delle immagini di train)
    val_images_dir = os.path.join(images_dir, 'val2017')
    if not os.path.exists(val_images_dir):
        os.makedirs(val_images_dir)

    train_images = os.listdir(train_images_dir)
    random.shuffle(train_images)
    val_split_size = int(0.2 * len(train_images))

    # Sposta il 20% delle immagini di train nella cartella val
    for image in train_images[:val_split_size]:
        shutil.move(
            os.path.join(train_images_dir, image),
            os.path.join(val_images_dir, image)
        )
    print(f"Creato dataset di validazione in {val_images_dir} con {val_split_size} immagini.")

    # Scarica le annotazioni
    download_and_extract(urls['annotations'], train_dataset_dir, train_dataset_dir)

    # Rinomina ed organizza le annotazioni
    annotations_zip_dir = os.path.join(train_dataset_dir, 'annotations')
    annotations_files = os.listdir(annotations_zip_dir)

    # Sposta i file .json nella cartella 'annotations'
    for file_name in annotations_files:
        if file_name.endswith('.json'):
            os.rename(
                os.path.join(annotations_zip_dir, file_name),
                os.path.join(annotations_dir, file_name)
            )

    print("Scaricamento e organizzazione del dataset COCO 2017 completati!")
else:
    print(f"Il dataset esiste già nella cartella {train_dataset_dir}. Saltando il download e l'estrazione.")


Il dataset esiste già nella cartella C:/Users/Tesisti/Desktop/CalabreseCarretta/Progetto\COCOdataset2017. Saltando il download e l'estrazione.


5 - DEFINIZIONE DELLE FUNZIONI PER CARICARE IMMAGINI E MASCHERE NEI DATASET TENSORFLOW

In [6]:
dim = 128

def get_image_id_from_path(image_path, coco):
    """ Cerca l'ID dell'immagine nel dataset COCO basato sul nome del file """
    file_name = os.path.basename(image_path)
    
    # Ottieni tutte le immagini dal dataset COCO
    img_info_list = coco.loadImgs(coco.getImgIds())
    
    # Cerca l'ID immagine corrispondente al nome del file
    for img_info in img_info_list:
        if img_info['file_name'] == file_name:
            return img_info['id']
    
    # Se l'immagine non è stata trovata, solleva un'eccezione
    raise ValueError(f"ID immagine non trovato per il file: {file_name}")

def create_mask(image_id, coco):
    annotation_ids = coco.getAnnIds(imgIds=image_id, iscrowd=False)
    annotations = coco.loadAnns(annotation_ids)
    #print(f"Trovate {len(annotations)} annotazioni per l'immagine {image_id}")
    
    mask = np.zeros((dim, dim), dtype=np.uint8)
    for annotation in annotations:
        annotation_mask = coco.annToMask(annotation)
        resized_mask = cv2.resize(annotation_mask, (dim, dim), interpolation=cv2.INTER_NEAREST)
        mask = np.maximum(mask, resized_mask)
    return mask

def load_image(image_path):
    # Carica e decodifica l'immagine
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    
    # Ridimensiona l'immagine a dimxdim
    image = tf.image.resize(image, [dim, dim])
    
    # Normalizza l'immagine nel range [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    
    return image

def load_image_with_mask(image_path, coco_key):
    """Carica l'immagine e genera la maschera corrispondente."""
    # Converti il tensore in una stringa se necessario
    image_path_str = image_path.numpy().decode('utf-8') if hasattr(image_path, 'numpy') else image_path
    coco_key_str = coco_key.numpy().decode('utf-8') if hasattr(coco_key, 'numpy') else coco_key

    # Controlla se il file esiste
    if not os.path.exists(image_path_str):
        print(f"Immagine non trovata per il file: {image_path_str}")
        empty_image = tf.zeros((dim, dim, 3), dtype=tf.float32)
        empty_mask = tf.zeros((dim, dim, 1), dtype=tf.float32)
        return empty_image, empty_mask

    try:
        # Carica l'immagine
        image = load_image(image_path_str)
        image_id = get_image_id_from_path(image_path_str, coco_dict[coco_key_str])

        # Crea la maschera associata
        mask = create_mask(image_id, coco_dict[coco_key_str])

        # Converti la maschera in un tensore di TensorFlow
        mask = tf.convert_to_tensor(mask, dtype=tf.float32)
        mask = tf.expand_dims(mask, axis=-1)  # Aggiungi una dimensione per renderla compatibile con (dim, dim, 1)

        return image, mask

    except Exception as e:
        print(f"Errore durante il caricamento di immagine o maschera: {e}")
        empty_image = tf.zeros((dim, dim, 3), dtype=tf.float32)
        empty_mask = tf.zeros((dim, dim, 1), dtype=tf.float32)
        return empty_image, empty_mask

def load_image_wrapper(image_path_tensor, coco_key_tensor):
    """Wrapper per usare tf.py_function nella pipeline TensorFlow."""
    image, mask = tf.py_function(func=load_image_with_mask,
                                 inp=[image_path_tensor, coco_key_tensor],
                                 Tout=[tf.float32, tf.float32])
    
    # Definisci le dimensioni e le forme per TensorFlow
    image.set_shape([dim, dim, 3])
    mask.set_shape([dim, dim, 1])
    return image, mask

# Definizione del livello di data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    ])
# Funzione di augmentazione che applica le stesse trasformazioni
def augment_image_and_mask(image, mask):
    # Concatenare immagine e maschera per applicare la stessa trasformazione
    concatenated = tf.concat([image, mask], axis=-1)
        
    # Applicare data augmentation su immagine + maschera concatenate
    augmented = data_augmentation(concatenated)
        
    # Separare immagine e maschera
    augmented_image = augmented[..., :-1]  # Canali immagine
    augmented_mask = augmented[..., -1:]  # Canale maschera (interpolazione nearest)
        
    return augmented_image, augmented_mask
    
# Applicare l'augmentazione al dataset
def apply_augmentation_to_dataset(dataset):
    return dataset.map(lambda image, mask: augment_image_and_mask(image, mask),
                           num_parallel_calls=tf.data.AUTOTUNE)

6 - DEFINIZIONI FUNZIONI PER OTTENERE UNA PARTE DEL DATASET E PER PREPROCESSARE IL DATASET

In [7]:
# Funzioni per ottenere un sottoinsieme del dataset e preprocessarlo
def get_small_dataset(dataset, fraction = 0.1):
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()
    small_dataset_size = int(dataset_size * fraction)
    return dataset.take(small_dataset_size)

def preprocess_dataset(dataset):
    def preprocess(x, y=None):
        x = tf.cast(x, tf.float32)
        if y is not None:  # Se y è presente, preprocessiamo anche la maschera
            y = tf.cast(y, tf.float32)
            return x, y
        else:
            return x  # Se y non è presente, restituiamo solo x

    return dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

7 - CREAZIONI DATASET TENSORFLOW TRAIN, VAL E TEST

In [8]:
coco_annotation_path = os.path.join(train_dataset_dir, 'annotations', 'instances_train2017.json')
# Caricare annotazioni COCO per il set di addestramento
annotation_coco = COCO(coco_annotation_path)

# Dizionario per memorizzare gli oggetti COCO
coco_dict = {'annotation': annotation_coco}

# Directory delle immagini
train_images_dir = os.path.join(train_dataset_dir, 'images', 'train2017')
val_images_dir = os.path.join(train_dataset_dir, 'images', 'val2017')

# Verifica i file nelle cartelle
train_image_paths = [os.path.join(train_images_dir, fname) for fname in os.listdir(train_images_dir) if fname.endswith(('.jpg', '.jpeg', '.png'))]
val_image_paths = [os.path.join(val_images_dir, fname) for fname in os.listdir(val_images_dir) if fname.endswith(('.jpg', '.jpeg', '.png'))]

# Creazione dei dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, ['annotation'] * len(train_image_paths)))
train_dataset = train_dataset.map(lambda x, y: load_image_wrapper(x, y), num_parallel_calls=tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_image_paths, ['annotation'] * len(val_image_paths)))
val_dataset = val_dataset.map(lambda x, y: load_image_wrapper(x, y), num_parallel_calls=tf.data.AUTOTUNE)

# Applica batching e prefetching
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Esegui l'addestramento su un sottoinsieme ridotto del dataset
train_dataset = preprocess_dataset(train_dataset)
val_dataset = preprocess_dataset(val_dataset)

small_train_dataset = preprocess_dataset(get_small_dataset(train_dataset))
small_val_dataset = preprocess_dataset(get_small_dataset(val_dataset))

test_image_paths = [os.path.join(test_images_dir, fname) for fname in os.listdir(test_images_dir) if fname.endswith(('.jpg', '.jpeg', '.png'))]

test_dataset = tf.data.Dataset.from_tensor_slices(test_image_paths)
test_dataset = test_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = preprocess_dataset(test_dataset)

loading annotations into memory...
Done (t=27.61s)
creating index...
index created!


In [ ]:
#CANCELLARE LA CELLA
small_train_dataset = preprocess_dataset(get_small_dataset(train_dataset))
small_val_dataset = preprocess_dataset(get_small_dataset(val_dataset))

8 - DEFINIZIONE DEL MODELLO CON VGG

In [9]:
# Blocco di attenzione

def attention_block_with_se(x, g, inter_channel):
    theta_x = layers.Conv2D(inter_channel, 1)(x)
    phi_g = layers.Conv2D(inter_channel, 1)(g)
    concat_xg = layers.add([theta_x, phi_g])
    relu_xg = layers.Activation('relu')(concat_xg)
    psi = layers.Conv2D(1, 1, activation='sigmoid')(relu_xg)

    # Squeeze-and-Excitation block
    se_shape = (1, 1, x.shape[-1])
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Reshape(se_shape)(se)
    se = layers.Dense(inter_channel // 2, activation='relu')(se)
    se = layers.Dense(x.shape[-1], activation='sigmoid')(se)
    x_se = layers.multiply([x, se])

    return layers.multiply([x_se, psi])


def create_unet_model_with_vgg16(input_shape=(dim, dim, 3)):
    vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    
    for layer in vgg16.layers:
        layer.trainable = False

    inputs = layers.Input(shape=input_shape)
    
    conv1 = vgg16.get_layer('block1_conv2')(vgg16.get_layer('block1_conv1')(inputs))
    conv1 = layers.BatchNormalization()(conv1)
    pool1 = vgg16.get_layer('block1_pool')(conv1)

    conv2 = vgg16.get_layer('block2_conv2')(vgg16.get_layer('block2_conv1')(pool1))
    conv2 = layers.BatchNormalization()(conv2)
    pool2 = vgg16.get_layer('block2_pool')(conv2)

    conv3 = vgg16.get_layer('block3_conv3')(vgg16.get_layer('block3_conv2')(vgg16.get_layer('block3_conv1')(pool2)))
    conv3 = layers.BatchNormalization()(conv3)
    pool3 = vgg16.get_layer('block3_pool')(conv3)

    conv4 = vgg16.get_layer('block4_conv3')(vgg16.get_layer('block4_conv2')(vgg16.get_layer('block4_conv1')(pool3)))
    drop4 = layers.SpatialDropout2D(0.5)(conv4)
    pool4 = vgg16.get_layer('block4_pool')(drop4)

    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same', dilation_rate=2, kernel_regularizer=regularizers.l2(0.0002))(pool4)
    conv5 = layers.BatchNormalization()(conv5)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0002))(conv5)
    conv5 = layers.BatchNormalization()(conv5)
    drop5 = layers.SpatialDropout2D(0.5)(conv5)

    up6 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(drop5))
    merge6 = layers.concatenate([drop4, up6], axis=3)
    conv6 = layers.DepthwiseConv2D(3, padding='same')(merge6)
    conv6 = layers.Conv2D(512, 1, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(conv6)
    conv6 = layers.BatchNormalization()(conv6)
    conv6 = layers.Dropout(0.4)(conv6)

    attention6 = attention_block_with_se(conv6, up6, 512)

    up7 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(attention6))
    merge7 = layers.concatenate([conv3, up7], axis=3)
    conv7 = layers.DepthwiseConv2D(3, padding='same')(merge7)
    conv7 = layers.Conv2D(512, 1, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(conv7)
    conv7 = layers.BatchNormalization()(conv7)
    conv7 = layers.Dropout(0.4)(conv7)

    attention7 = attention_block_with_se(conv7, up7, 512)

    up8 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(attention7))
    merge8 = layers.concatenate([conv2, up8], axis=3)
    conv8 = layers.DepthwiseConv2D(3, padding='same')(merge8)
    conv8 = layers.Conv2D(256, 1, activation='relu', kernel_regularizer=regularizers.l2(0.00005))(conv8)
    conv8 = layers.BatchNormalization()(conv8)
    conv8 = layers.Dropout(0.3)(conv8)

    attention8 = attention_block_with_se(conv8, up8, 256)

    up9 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(attention8))
    merge9 = layers.concatenate([conv1, up9], axis=3)
    conv9 = layers.DepthwiseConv2D(3, padding='same')(merge9)
    conv9 = layers.Conv2D(64, 1, activation='relu', kernel_regularizer=regularizers.l2(0.00005))(conv9)
    conv9 = layers.BatchNormalization()(conv9)
    conv9 = layers.Dropout(0.3)(conv9)

    attention9 = attention_block_with_se(conv9, up9, 64)
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(attention9)

    model = Model(inputs, outputs)
    return model

9 - DEFINIZIONE MODELLO CON RESNET

In [10]:
@register_keras_serializable()
class ResizeLayer(layers.Layer):
    def __init__(self, target_height, target_width, **kwargs):
        super(ResizeLayer, self).__init__(**kwargs)
        self.target_height = target_height
        self.target_width = target_width

    def call(self, inputs):
        return tf.image.resize(inputs, size=(self.target_height, self.target_width), method='bilinear')

@register_keras_serializable()
class AdjustDimensionsLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(AdjustDimensionsLayer, self).__init__(**kwargs)

    def call(self, inputs):
        encoder_output, decoder_output = inputs
        encoder_shape = tf.shape(encoder_output)
        resized_decoder_output = tf.image.resize(
            decoder_output, size=(encoder_shape[1], encoder_shape[2]), method='bilinear'
        )
        return encoder_output, resized_decoder_output

def create_unet_model_with_resnet50(input_shape=(128, 128, 3)):
    resnet50 = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in resnet50.layers:
        layer.trainable = False

    inputs = layers.Input(shape=input_shape)

    # Estrarre feature intermedie da ResNet-50
    x = resnet50.get_layer('conv1_relu')(resnet50.get_layer('conv1_conv')(inputs))
    x = resnet50.get_layer('pool1_pool')(x)
    conv1 = resnet50.get_layer('conv2_block3_out')(x)  # (None, 64, 64, 256)
    conv2 = resnet50.get_layer('conv3_block4_out')(conv1)  # (None, 32, 32, 512)
    conv3 = resnet50.get_layer('conv4_block6_out')(conv2)  # (None, 16, 16, 1024)
    conv4 = resnet50.get_layer('conv5_block3_out')(conv3)  # (None, 8, 8, 2048)
    drop4 = layers.SpatialDropout2D(0.5)(conv4)

    # Decoder
    up6 = layers.Conv2D(1024, 2, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0001))(layers.UpSampling2D(size=(2, 2))(drop4))
    conv3, up6 = AdjustDimensionsLayer()([conv3, up6])
    merge6 = layers.concatenate([conv3, up6], axis=3)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0001))(merge6)
    conv6 = layers.Dropout(0.4)(conv6)

    up7 = layers.Conv2D(512, 2, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0001))(layers.UpSampling2D(size=(2, 2))(conv6))
    conv2, up7 = AdjustDimensionsLayer()([conv2, up7])
    merge7 = layers.concatenate([conv2, up7], axis=3)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0001))(merge7)
    conv7 = layers.Dropout(0.4)(conv7)

    up8 = layers.Conv2D(256, 2, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0001))(layers.UpSampling2D(size=(2, 2))(conv7))
    conv1, up8 = AdjustDimensionsLayer()([conv1, up8])
    merge8 = layers.concatenate([conv1, up8], axis=3)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0001))(merge8)
    conv8 = layers.Dropout(0.3)(conv8)

    up9 = layers.Conv2D(128, 2, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0001))(layers.UpSampling2D(size=(2, 2))(conv8))
    up9 = ResizeLayer(128, 128)(up9) if inputs.shape[1] != up9.shape[1] else up9
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.0001))(up9)
    conv9 = layers.Dropout(0.3)(conv9)

    # Ultimo layer per ottenere l'output di dimensione 128x128
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv9)

    model = models.Model(inputs, outputs)
    return model

#model.summary()


10 - DEFINIZIONE DELLE METRICHE

In [11]:
# Funzione personalizzata per SSIM, IoU e F1 Score e psnr
def ssim_metric(y_true, y_pred):
    return tf.image.ssim(y_true, y_pred, max_val=1.0)

def iou_metric(y_true, y_pred):
    y_pred = tf.round(y_pred)  # Arrotondamento a 0 o 1 per segmentazione binaria
    intersection = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
    union = tf.reduce_sum(tf.cast(y_true + y_pred - y_true * y_pred, tf.float32))
    iou = (intersection + 1e-10) / (union + 1e-10)  # Evita divisioni per zero
    return iou

def f1_score(y_true, y_pred):
    y_pred = tf.round(y_pred)  # Arrotondamento a 0 o 1 per segmentazione binaria
    intersection = tf.reduce_sum(y_true * y_pred)
    precision = intersection / (tf.reduce_sum(y_pred) + 1e-10)
    recall = intersection / (tf.reduce_sum(y_true) + 1e-10)
    f1 = (2 * precision * recall) / (precision + recall + 1e-10)  # Evita divisioni per zero
    return f1

def psnr_metric(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

11 - CREAZIONE, COMPILAZIONE E SALVATAGGIO DEL MODELLO 

In [22]:
# Percorsi in cui salvare o da cui caricare il modello
model_dir = os.path.join(project_dir, 'Modello')
model_resnet_path = os.path.join(model_dir, 'modello_resnet.keras')
model_vgg_path = os.path.join(model_dir, 'modello_vgg.keras')

@keras.saving.register_keras_serializable()
def bce_dice_loss(y_true, y_pred):
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice_loss = 1 - (2 * tf.reduce_sum(y_true * y_pred) + 1) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1)
    return bce_loss + dice_loss

@keras.saving.register_keras_serializable()
def bce_dice_loss_with_alpha(alpha):
    @keras.saving.register_keras_serializable()
    def loss(y_true, y_pred):
        bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        dice_loss = 1 - (2 * tf.reduce_sum(y_true * y_pred) + 1) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1)
        return alpha * bce_loss + (1 - alpha) * dice_loss
    return loss

class DynamicAlphaCallback(tf.keras.callbacks.Callback):
    def __init__(self, initial_alpha=0.5, final_alpha=0.1, total_epochs=20):
        super().__init__()
        self.initial_alpha = initial_alpha
        self.final_alpha = final_alpha
        self.total_epochs = total_epochs
        self.current_alpha = initial_alpha

    def on_epoch_begin(self, epoch, logs=None):
        # Aggiorna alpha in base all'epoca corrente
        self.current_alpha = self.initial_alpha - ((self.initial_alpha - self.final_alpha) * (epoch / self.total_epochs))
        print(f"\n[DynamicAlphaCallback] Epoch {epoch + 1}: Current alpha = {self.current_alpha:.4f}")
        # Aggiorna la loss nel modello
        self.model.loss = bce_dice_loss_with_alpha(self.current_alpha)


dynamic_alpha_callback = DynamicAlphaCallback(initial_alpha=0.5, final_alpha=0.1, total_epochs=20)

#Compila il modello con un valore iniziale per alpha
initial_alpha = 0.5

def training(model, model_path, augmentazione=True):
    
    # Compilazione del modello con le metriche personalizzate
    model.compile(
        optimizer=AdamW(learning_rate=0.0001, weight_decay=1e-5),
        loss=bce_dice_loss_with_alpha(initial_alpha),
        metrics=['accuracy']
    )

    # Callback per Early Stopping e riduzione del learning rate
    early_stopping = EarlyStopping(monitor='val_loss', 
                                   patience=5, 
                                   restore_best_weights=True)

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',        
        factor=0.5,               
        patience=3,               
        min_lr=1e-6                
    )

    if augmentazione:
        small_train_dataset = apply_augmentation_to_dataset(small_train_dataset)
    
    # Addestramento del modello
    history = model.fit(
        small_train_dataset,
        validation_data=small_val_dataset,
        epochs=10,
        verbose=1,
        callbacks=[early_stopping, reduce_lr, dynamic_alpha_callback]
    )

    # Estrazione delle metriche dalla history
    history_dict = history.history
    train_loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    # Salvare le metriche in un file JSON
    metriche_history = os.path.join(os.path.dirname(model_path), 'metriche_history.json')
    with open(metriche_history, 'w') as f:
        json.dump({'train_loss': train_loss, 'val_loss': val_loss}, f)

    # Salva il modello addestrato
    model.save(model_path)


if addestramento:
    # Creazione del modello
    model_vgg = create_unet_model_with_vgg16(input_shape=(dim, dim, 3))
    model_resnet = create_unet_model_with_resnet50(input_shape=(dim, dim, 3))

    training(model_vgg, model_vgg_path, augmentazione=True)
    #training(model_resnet, model_resnet_path, augmentazione=False)
       # Grafico dell'andamento di train_loss e val_loss
    plt.figure(figsize=(10, 6))
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()    
else: 
    # Caricamento del modello
    model = keras.models.load_model(
        model_vgg_path,
        custom_objects={
            #'bce_dice_loss_with_alpha': bce_dice_loss_with_alpha,  # Funzione di perdita principale
            'bce_dice_loss': bce_dice_loss,                       # Versione semplice della funzione di perdita
            'loss': bce_dice_loss_with_alpha(0.5),
            'psnr_metric' : psnr_metric# Funzione `loss` con un valore predefinito di alpha
        }
    )

    # model = keras.models.load_model(model_resnet_path, custom_objects={'bce_dice_loss_with_alpha': bce_dice_loss_with_alpha, 
    #                                                                    'ResizeLayer' : ResizeLayer,
    #                                                                     'bce_dice_loss' :bce_dice_loss ,
    #                                                                     'loss': bce_dice_loss_with_alpha(0.5),
    #                                                                     'AdjustDimensionsLayer' : AdjustDimensionsLayer,
    #                                                                    'psnr_metric' : psnr_metric})
    model.trainable = False

TEST PROCESSING

12 - VALUTAZIONE MODELLO

In [ ]:
def evaluate_metrics(model, dataset, save_path='metriche_tabella.png'):
 
    # Inizializza liste per raccogliere i valori delle metriche
    accuracy_values = []
    iou_values = []
    f1_values = []
    ssim_values = []
    psnr_values = []

    # Itera su ogni batch nel dataset di test
    for x, y_true in dataset:
        y_pred = model.predict(x, verbose=0)  # Previsione del modello

        # Calcola le metriche per ogni batch
        batch_accuracy = tf.reduce_mean(tf.keras.metrics.binary_accuracy(y_true, y_pred)).numpy()
        batch_iou = iou_metric(y_true, y_pred).numpy()  # Assumi che restituisca un valore scalare
        batch_f1 = f1_score(y_true, y_pred).numpy()  # Assumi che restituisca un valore scalare
        batch_ssim = ssim_metric(y_true, y_pred).numpy()  # Valore medio di SSIM
        batch_psnr = psnr_metric(y_true, y_pred).numpy()  # Valore medio di PSNR

        # Aggiunge i valori delle metriche alla lista
        accuracy_values.append(batch_accuracy)
        iou_values.append(batch_iou)
        f1_values.append(batch_f1)
        ssim_values.append(batch_ssim)
        psnr_values.append(batch_psnr)

    # Calcola media e massimo per ciascuna metrica
    metrics_summary = {
        'accuracy': {'mean': np.mean(accuracy_values), 'max': np.max(accuracy_values)},
        'iou': {'mean': np.mean(iou_values), 'max': np.max(iou_values)},
        'f1_score': {'mean': np.mean(f1_values), 'max': np.max(f1_values)},
        'ssim': {'mean': np.mean(ssim_values), 'max': np.max(ssim_values)},
        'psnr': {'mean': np.mean(psnr_values), 'max': np.max(psnr_values)},
    }

    # Creazione di una tabella con matplotlib
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.axis('tight')
    ax.axis('off')
    table_data = [
        ["Metric", "Mean", "Max"],
        ["Accuracy", f"{metrics_summary['accuracy']['mean']:.4f}", f"{metrics_summary['accuracy']['max']:.4f}"],
        ["IoU", f"{metrics_summary['iou']['mean']:.4f}", f"{metrics_summary['iou']['max']:.4f}"],
        ["F1 Score", f"{metrics_summary['f1_score']['mean']:.4f}", f"{metrics_summary['f1_score']['max']:.4f}"],
        ["SSIM", f"{metrics_summary['ssim']['mean']:.4f}", f"{metrics_summary['ssim']['max']:.4f}"],
        ["PSNR", f"{metrics_summary['psnr']['mean']:.4f}", f"{metrics_summary['psnr']['max']:.4f}"]
    ]
    table = ax.table(cellText=table_data, colLabels=None, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Salva l'immagine
    plt.savefig(save_path, bbox_inches='tight')
    plt.close(fig)

    return metrics_summary

metriche_tabella_train = os.path.join(model_dir, 'metriche_tabella_train.png')
metriche_tabella_val = os.path.join(model_dir, 'metriche_tabella_val.png')
metrics = evaluate_metrics(model, small_train_dataset, save_path=metriche_tabella_train)
metrics = evaluate_metrics(model, small_val_dataset, save_path=metriche_tabella_val)
print(metrics)

13 - DEFINIZIONE FUNZIONI PER OTTENERE LA DEPTH MAP E LE POINT CLOUD

In [ ]:
# Funzione per caricare il modello MiDaS v3
def load_midas_model():
    model_type = "DPT_Large"  # Usa DPT_Large per maggiore accuratezza
    midas = torch.hub.load("intel-isl/MiDaS", model_type)
    midas.eval()
    transform = torch.hub.load("intel-isl/MiDaS", "transforms").dpt_transform
    return midas, transform

# Funzione per ottenere la mappa di profondità utilizzando MiDaS v3
def get_depth_map(image, midas, transform):
    img_np = (image.numpy() * 255).astype(np.uint8)

    if img_np.shape[-1] == 3:
        img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

    input_tensor = transform(img_np).unsqueeze(0)
    if input_tensor.dim() == 5:
        input_tensor = input_tensor.squeeze(1)

    with torch.no_grad():
        depth_map = midas(input_tensor).squeeze().cpu().numpy()

    depth_map_normalized = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min()) * 255
    depth_map_normalized = depth_map_normalized.astype(np.uint8)

    depth_map_filtered = median(depth_map_normalized, disk(3))
    depth_map_filtered = depth_map_filtered.astype(np.uint8)

    return depth_map_filtered

# Funzione per creare la nuvola di punti 3D usando la mappa di profondità
def image_to_3d_point_cloud(depth_map, intrinsics):
    height, width = depth_map.shape
    y_indices, x_indices = np.indices((height, width))
    z_values = depth_map[y_indices, x_indices]

    f_x = intrinsics['f_x']
    f_y = intrinsics['f_y']
    c_x = intrinsics['c_x']
    c_y = intrinsics['c_y']

    X = (x_indices - c_x)
    Y = (y_indices - c_y)
    Z = z_values

    point_cloud_3d = np.column_stack((X.flatten(), Y.flatten(), Z.flatten()))
    return point_cloud_3d

# Parametri intrinseci della fotocamera
intrinsics = {
    'f_x': dim,  # Lunghezza focale sull'asse X
    'f_y': dim,  # Lunghezza focale sull'asse Y
    'c_x': 64,   # Centro ottico sull'asse X
    'c_y': 64    # Centro ottico sull'asse Y
}

14 - DEFINIZIONE FUNZIONE PER VISUALIZZARE IMMAGINE, MASCHERA ORIGINALE, MASCHERA PREDETTA, DEPTH MAP, POINT CLOUD E POINT CLOUD SEGMENTATA

In [ ]:
def visualize_and_save_predictions_with_depth(model, dataset, midas, transform, intrinsics, num_samples=3, max_points=100000, has_masks=True, output_dir='./output'):
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    dataset_iter = iter(dataset)
    samples_shown = 0

    print("Starting image processing and saving...")

    for batch in dataset_iter:
        if samples_shown >= num_samples:
            break

        if has_masks:
            images, masks = batch
        else:
            images = batch

        for j in range(images.shape[0]):
            if samples_shown >= num_samples:
                break

            image = images[j]
            if has_masks:
                mask = masks[j]

            # Perform prediction
            pred = model.predict(tf.expand_dims(image, axis=0))
            binary_pred = (pred > 0.5).astype(np.uint8).squeeze()

            # Get the depth map
            depth_map = get_depth_map(image, midas, transform)

            # Estrai la nuvola di punti 3D
            point_cloud_3d = image_to_3d_point_cloud(depth_map, intrinsics)

            # Visualize and save all in a single plot
            try:
                plt.figure(figsize=(20, 10))
                
                # Original Image
                plt.subplot(1, 4, 1)
                plt.imshow((image.numpy() * 255).astype("uint8"))
                plt.title("Original Image")
                plt.axis("off")

                # Original Mask, if present
                if has_masks:
                    plt.subplot(1, 4, 2)
                    plt.imshow((image.numpy() * 255).astype("uint8"))
                    plt.imshow(mask.numpy().squeeze(), cmap='jet', alpha=0.5)
                    plt.title("Original Mask")
                    plt.axis("off")

                # Predicted Mask
                plt.subplot(1, 4, 3)
                plt.imshow((image.numpy() * 255).astype("uint8"))
                plt.imshow(binary_pred, cmap='jet', alpha=0.5)
                plt.title("Predicted Mask")
                plt.axis("off")

                # Depth Map
                plt.subplot(1, 4, 4)
                plt.imshow(depth_map, cmap='gray')
                plt.title("Depth Map")
                plt.axis("off")

                # Save combined figure
                combined_path = f"{output_dir}/image_{samples_shown}_combined.png"
                plt.savefig(combined_path)
                print(f"Combined image saved: {combined_path}")
                plt.show()

                 # Visualizza e salva la nuvola di punti 3D non segmentata
                fig_original = go.Figure(data=[go.Scatter3d(
                    x=point_cloud_3d[:, 0],
                    y=point_cloud_3d[:, 1],
                    z=point_cloud_3d[:, 2],
                    mode='markers',
                    marker=dict(
                        size=2,
                        color=point_cloud_3d[:, 2],
                        colorscale='Jet',
                        opacity=0.5
                    )
                )])
                fig_original.update_layout(
                    scene=dict(
                        xaxis_title='X coordinates',
                        yaxis_title='Y coordinates',
                        zaxis_title='Depth (Z)'
                    ),
                    title="Interactive 3D Point Cloud (No Segmentation)"
                )
                html_path = f"{output_dir}/point_cloud_{samples_shown}_original.html"
                fig_original.write_html(html_path)
                print(f"Nuvola di punti originale salvata: {html_path}")
                fig_original.show()  # Mostra la nuvola non segmentata in Jupyter

                # Ridimensiona la maschera predetta per corrispondere alla risoluzione della mappa di profondità
                binary_pred_resized = cv2.resize(binary_pred, (depth_map.shape[1], depth_map.shape[0]), interpolation=cv2.INTER_NEAREST)
                binary_pred_flat = binary_pred_resized.flatten()

                # Colori: rosso per l'oggetto (maschera == 1), blu per il resto (maschera == 0)
                colors = np.where(binary_pred_flat == 1, 'red', 'blue')

                # Visualizza e salva la nuvola di punti 3D segmentata
                fig_segmented = go.Figure(data=[go.Scatter3d(
                    x=point_cloud_3d[:, 0],
                    y=point_cloud_3d[:, 1],
                    z=point_cloud_3d[:, 2],
                    mode='markers',
                    marker=dict(
                        size=2,
                        color=colors,
                        opacity=0.8
                    )
                )])

                fig_segmented.update_layout(
                    scene=dict(
                        xaxis_title='X coordinates',
                        yaxis_title='Y coordinates',
                        zaxis_title='Depth (Z)'
                    ),
                    title="Interactive 3D Point Cloud with Object Coloring"
                )

                # Salva la nuvola segmentata come HTML
                html_segmented_path = f"{output_dir}/point_cloud_{samples_shown}_segmented.html"
                fig_segmented.write_html(html_segmented_path)
                print(f"Nuvola di punti segmentata salvata: {html_segmented_path}")
                fig_segmented.show()  # Mostra la nuvola segmentata in Jupyter

                # Increment sample count
                samples_shown += 1

            except Exception as e:
                print(f"Error during saving and visualization: {e}")

            if samples_shown >= num_samples:
                break

15 - VISUALIZZAZIONE RISULTATI TRAINING

In [ ]:
# Carica il modello MiDaS v3
midas, transform = load_midas_model()

# Esegui la visualizzazione con mappa di profondità e nuvola 3D
visualize_and_save_predictions_with_depth(model, small_train_dataset, midas, transform, intrinsics, num_samples=25, max_points=100000, has_masks=True)

16 - VISUALIZZAZIONE RISULTATI DATASET DI TEST

In [ ]:
visualize_and_save_predictions_with_depth(model, test_dataset, midas, transform, intrinsics, num_samples=30, max_points=100000, has_masks=False)

17 TEST CON IMMAGINE SCELTA DA PC

In [ ]:
# Funzione per selezionare l'immagine dal PC
def select_image():
    # Usa un file dialog per scegliere un'immagine
    root = Tk()
    root.withdraw()  # Nasconde la finestra principale di Tkinter
    root.attributes("-topmost", 1)  # Porta la finestra in primo piano
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp")])
    root.destroy()
    
    if not file_path:
        print("Nessun file selezionato.")
        return None

    # Carica l'immagine usando la funzione load_image
    return load_image(file_path)

# Carica l'immagine
image_tensor = select_image()

if image_tensor is not None:
    # Aggiungi la dimensione del batch
    image_tensor = tf.expand_dims(image_tensor, axis=0)

if image_tensor is not None:
    # Crea un dataset simulato con una singola immagine per compatibilità con la funzione
    dataset = tf.data.Dataset.from_tensors(image_tensor)

    # Chiama la funzione di visualizzazione con questo dataset a immagine singola
    output_dir='./output'
    visualize_and_save_predictions_with_depth(
        model=model,
        dataset=dataset,
        midas=midas,
        transform=transform,
        intrinsics=intrinsics,
        num_samples=1,  # Una singola immagine
        max_points=100000, has_masks=False
    )
else:
    print("Nessuna immagine è stata selezionata o caricata.")